In [1]:
import glob
import os

import astropy.units as u
from astropy.table import Table, Column
from astropy.io.votable.tree import Param,Info
from astropy.io.votable import from_table, writeto
import matplotlib.pyplot as plt
import numpy as np

from RadioAbsTools import cube_tools, spectrum_tools



In [2]:
folder = 'sb8906/casda/'
file_list = glob.glob(folder + 'sb8906_0029-7228_sl_*K.fits')
file_list.sort()
file_list

['sb8906/casda/sb8906_0029-7228_sl_0.8K.fits',
 'sb8906/casda/sb8906_0029-7228_sl_1.0K.fits',
 'sb8906/casda/sb8906_0029-7228_sl_1.2K.fits',
 'sb8906/casda/sb8906_0029-7228_sl_1.4K.fits',
 'sb8906/casda/sb8906_0029-7228_sl_1.6K.fits',
 'sb8906/casda/sb8906_0029-7228_sl_1.8K.fits',
 'sb8906/casda/sb8906_0029-7228_sl_2.0K.fits']

In [3]:
def output_spectra(velocity, opacity, flux, filename, 
                   sigma_tau):
    """
    Write the spectrum (velocity, flux and opacity) to a votable format file.

    :param spectrum: The spectrum to be output.
    :param opacity:  The opacity to be output.
    :param filename:  The filename to be created
    :param longitude: The galactic longitude of the target object
    :param latitude: The galactic latitude of the target object
    """
    table = Table(meta={'name': filename, 'id': 'opacity'})
    table.add_column(Column(name='velocity', data=velocity, unit='m/s'))
    table.add_column(Column(name='opacity', data=opacity))
    table.add_column(Column(name='flux', data=spectrum.flux, unit='Jy', description='Flux per beam'))
    table.add_column(Column(name='sigma_opacity', data=sigma_tau, description='Noise in the absorption profile'))

    votable = from_table(table)
    writeto(votable, filename)



In [4]:
# run agean to identify sources (or use preexisting)
# identify continuum velocity ranges
start_vel=(-90*u.km).to(u.m).value
end_vel=(-30*u.km).to(u.m).value

# Note that we need to make sure the beam info is set:
# sethead BMAJ='0.0047402' BMIN='0.0040442' BPA='83.183' sb8906/casda/sb8906_0029-7228_sl_*K.fits

# loop through and extract spectra for each

In [5]:
sd_list = {}
max_list = {}
for name in file_list:
    spec, source_ids, islands = cube_tools.extract_spectra(fits_filename=name, 
                      isle_filename='0029-7228_src_isle.vot', src_filename='0029-7228_src_comp.vot',
                      continuum_start_vel=start_vel, continuum_end_vel=end_vel)
    spectrum = list(spec.values())[0]
    spec_mean, spec_sd = spectrum_tools.get_mean_continuum(spectrum.velocity, spectrum.flux, start_vel, end_vel)
    sigma_tau = np.full(spectrum.velocity.shape, spec_sd)
    print (spec_mean, np.max(spectrum.flux))
    spectrum_tools.plot_absorption_spectrum(spectrum.velocity, spectrum.flux/spec_mean, name[:-5]+'_spec.png',
                                           name, start_vel, end_vel, sigma_tau, range=(-100,250))
    
    output_spectra(spectrum.velocity, spectrum.flux/spec_mean, spectrum.flux, name[:-5]+'_spec.xml', sigma_tau)
    #plt.plot(spectrum.velocity/1000, spectrum['flux'])
    #plt.title(name)
    #plt.savefig(name[:-5]+'_spec.png')
    #plt.close()
    sd_list[name] = spec_sd
    max_list[name] = np.max(spectrum.flux/spec_mean)

Extracting sources from 0029-7228_src_comp.vot
Found source 1-0 at 7.4825, -72.5393 with flux 0.0016 and rms of 0.0004 giving S/N of 4.4495
Ignoring source at 7.4825, -72.5393 due to low S/N of 4.4495 or flux of 0.0016
Found source 2-0 at 7.0831, -72.5325 with flux 0.0077 and rms of 0.0004 giving S/N of 21.4299
Ignoring source at 7.0831, -72.5325 due to low S/N of 21.4299 or flux of 0.0077
Found source 3-0 at 7.3288, -72.4698 with flux 0.0881 and rms of 0.0004 giving S/N of 210.7897
Extracting islands from 0029-7228_src_isle.vot
Using bins 11 to 70 (velocity range -90000 to -30000) out of 494
0.0019984378 0.0023623551


Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


Extracting sources from 0029-7228_src_comp.vot
Found source 1-0 at 7.4825, -72.5393 with flux 0.0016 and rms of 0.0004 giving S/N of 4.4495
Ignoring source at 7.4825, -72.5393 due to low S/N of 4.4495 or flux of 0.0016
Found source 2-0 at 7.0831, -72.5325 with flux 0.0077 and rms of 0.0004 giving S/N of 21.4299
Ignoring source at 7.0831, -72.5325 due to low S/N of 21.4299 or flux of 0.0077
Found source 3-0 at 7.3288, -72.4698 with flux 0.0881 and rms of 0.0004 giving S/N of 210.7897
Extracting islands from 0029-7228_src_isle.vot
Using bins 11 to 70 (velocity range -90000 to -30000) out of 494
0.0020002252 0.0023570436


Extracting sources from 0029-7228_src_comp.vot
Found source 1-0 at 7.4825, -72.5393 with flux 0.0016 and rms of 0.0004 giving S/N of 4.4495
Ignoring source at 7.4825, -72.5393 due to low S/N of 4.4495 or flux of 0.0016
Found source 2-0 at 7.0831, -72.5325 with flux 0.0077 and rms of 0.0004 giving S/N of 21.4299
Ignoring source at 7.0831, -72.5325 due to low S/N of 21.4299 or flux of 0.0077
Found source 3-0 at 7.3288, -72.4698 with flux 0.0881 and rms of 0.0004 giving S/N of 210.7897
Extracting islands from 0029-7228_src_isle.vot
Using bins 11 to 70 (velocity range -90000 to -30000) out of 494
0.0020002606 0.0023907118


Extracting sources from 0029-7228_src_comp.vot
Found source 1-0 at 7.4825, -72.5393 with flux 0.0016 and rms of 0.0004 giving S/N of 4.4495
Ignoring source at 7.4825, -72.5393 due to low S/N of 4.4495 or flux of 0.0016
Found source 2-0 at 7.0831, -72.5325 with flux 0.0077 and rms of 0.0004 giving S/N of 21.4299
Ignoring source at 7.0831, -72.5325 due to low S/N of 21.4299 or flux of 0.0077
Found source 3-0 at 7.3288, -72.4698 with flux 0.0881 and rms of 0.0004 giving S/N of 210.7897
Extracting islands from 0029-7228_src_isle.vot
Using bins 11 to 70 (velocity range -90000 to -30000) out of 494
0.001989125 0.00240137


Extracting sources from 0029-7228_src_comp.vot
Found source 1-0 at 7.4825, -72.5393 with flux 0.0016 and rms of 0.0004 giving S/N of 4.4495
Ignoring source at 7.4825, -72.5393 due to low S/N of 4.4495 or flux of 0.0016
Found source 2-0 at 7.0831, -72.5325 with flux 0.0077 and rms of 0.0004 giving S/N of 21.4299
Ignoring source at 7.0831, -72.5325 due to low S/N of 21.4299 or flux of 0.0077
Found source 3-0 at 7.3288, -72.4698 with flux 0.0881 and rms of 0.0004 giving S/N of 210.7897
Extracting islands from 0029-7228_src_isle.vot
Using bins 11 to 70 (velocity range -90000 to -30000) out of 494
0.0019886398 0.0023888962


Extracting sources from 0029-7228_src_comp.vot
Found source 1-0 at 7.4825, -72.5393 with flux 0.0016 and rms of 0.0004 giving S/N of 4.4495
Ignoring source at 7.4825, -72.5393 due to low S/N of 4.4495 or flux of 0.0016
Found source 2-0 at 7.0831, -72.5325 with flux 0.0077 and rms of 0.0004 giving S/N of 21.4299
Ignoring source at 7.0831, -72.5325 due to low S/N of 21.4299 or flux of 0.0077
Found source 3-0 at 7.3288, -72.4698 with flux 0.0881 and rms of 0.0004 giving S/N of 210.7897
Extracting islands from 0029-7228_src_isle.vot
Using bins 11 to 70 (velocity range -90000 to -30000) out of 494
0.0019871162 0.002391766


Extracting sources from 0029-7228_src_comp.vot
Found source 1-0 at 7.4825, -72.5393 with flux 0.0016 and rms of 0.0004 giving S/N of 4.4495
Ignoring source at 7.4825, -72.5393 due to low S/N of 4.4495 or flux of 0.0016
Found source 2-0 at 7.0831, -72.5325 with flux 0.0077 and rms of 0.0004 giving S/N of 21.4299
Ignoring source at 7.0831, -72.5325 due to low S/N of 21.4299 or flux of 0.0077
Found source 3-0 at 7.3288, -72.4698 with flux 0.0881 and rms of 0.0004 giving S/N of 210.7897
Extracting islands from 0029-7228_src_isle.vot
Using bins 11 to 70 (velocity range -90000 to -30000) out of 494
0.001972001 0.0023404656


In [ ]:
print (sd_list)

In [ ]:
uvdists = []
sds = []
maxs = []
pref_len = len(folder)
for key in sd_list.keys():
    uvdist = key[pref_len+20:-6]
    print  (uvdist)
    uvdists.append(float(uvdist))
    sds.append(sd_list[key])
    maxs.append(max_list[key])
    
plt.scatter(uvdists, sds)
plt.grid()
#plt.xlim((0,3))
#plt.ylim((0.075,0.095))
plt.xlabel(r'Minimum baseline length (kilo-$\lambda$ )')
plt.ylabel("Optical Depth Noise")
plt.savefig(folder+'uv-dist-noise-compare.png')
plt.close()

plt.scatter(uvdists, maxs)
plt.grid()
#plt.xlim((0,3))
#plt.ylim((0.075,0.095))
plt.xlabel(r'Minimum baseline length (kilo-$\lambda$ )')
plt.ylabel("Maxmium Optical Depth")
plt.savefig(folder+'uv-dist-max-compare.png')